# Interagindo visualmente com dados

Analisar visualmente um conjunto de dados é uma tarefa importante no processo de ciência de dados. Até aqui, vimos como fazer isso usando gráficos estáticos produzidos pelas bibliotecas `matplotlib` e `seaborn`. Neste notebook, vamos explorar a biblioteca de visualição interativa de dados [Plotly](https://plot.ly/python/), uma biblioteca compatível com diferentes linguagens de programação. Para o ecossistema Python, podemos usar o módulo `plotly.express`, criado para facilitar a produção de visualizações interativas.

In [0]:
import pandas as pd
import plotly.express as px

Para esta análise, vamos utilizar [uma base de dados aberta sobre preços de combustíveis](https://http://dados.gov.br/dataset/infopreco), disponibilizada pelos postos no sítio da ANP. No código abaixo, vamos informar ao Pandas que trate a característica `DATA CADASTRO` como uma data:

In [0]:
preços = pd.read_csv('http://www.anp.gov.br/images/infopreco/infopreco.csv', encoding='latin-1', sep=';', decimal=',', parse_dates=['DATA CADASTRO'])
preços.head()

,CNPJ,NOME,ENDEREÇO,COMPLEMENTO,BAIRRO,MUNICÍPIO,UF,PRODUTO,VALOR VENDA,DATA CADASTRO
0,62780000102,AUTO POSTO PARATI,"AVENIDA FILINTO MULLER,645",NaN,CENTRO,TRES LAGOAS,MS,Gasolina C Comum,4.436,2018-06-28 17:49:00
1,62780000102,AUTO POSTO PARATI,"AVENIDA FILINTO MULLER,645",NaN,CENTRO,TRES LAGOAS,MS,Etanol,3.482,2018-06-28 17:49:00
2,62780000102,AUTO POSTO PARATI,"AVENIDA FILINTO MULLER,645",NaN,CENTRO,TRES LAGOAS,MS,Diesel S500,3.644,2018-06-28 17:49:00
3,62780000102,AUTO POSTO PARATI,"AVENIDA FILINTO MULLER,645",NaN,CENTRO,TRES LAGOAS,MS,Diesel S10,3.734,2018-06-28 17:49:00
4,300357000195,POSTO E TRANSPORTADORA PEGORARO,"RODOVIA BR 163,S/N",KM 786,ZONA RURAL,COXIM,MS,Gasolina C Comum,4.349,2018-09-12 19:06:00


## Validando os dados

Vamos começar nossa análise validando os dados presentes na base. Isto é particularmente importante em bases de dados abertos governamentais, que muitas vezes não contam com boa curadoria de dados.

In [0]:
preços.isnull().sum()

CNPJ               0
NOME               0
ENDEREÇO           0
COMPLEMENTO      666
BAIRRO            11
MUNICÍPIO          0
UF                 0
PRODUTO            0
VALOR VENDA        0
DATA CADASTRO      0
dtype: int64

In [0]:
preço_produto = preços.pivot_table(index="PRODUTO", values="VALOR VENDA")
preço_produto.head()

,VALOR VENDA
PRODUTO,
Diesel S10,8.315152
Diesel S500,9.767784
Etanol,6.974361
GNV,18.362318
Gasolina C Comum,11.278889


Os dados faltantes se limitam a bairro e complemento, que não serão foco da nossa análise. Em compensação, a análise do preço médio dos combustíveis listados indica valores consideravelmente elevados. Isto é um indicativo da presença de valores incorretos, que podemos verificar analisando a distribuição dos dados. Para interagir com o gráfico, passe o cursor do mouse sobre o gráfico e explore as opções no canto superior direito:

In [0]:
px.box(preços, x="PRODUTO", y="VALOR VENDA")

Note que o gráfico produzido pelo Plotly traz um número de ferramentas que podemos usar para aprofundar nossa investigação. Vamos destacar alguns deles:

* Ao posicionar o mouse sobre um boxplot, vemos suas informação.
* Ao posicionar o mouse sobre um outlier, podemos ver seu valor.
* Podemos dar zoom sobre as partes do gráfico que mais nos interessem.
* Podemos salvar o gráfico diretamente.

Analisando especificamente os dados da base em questão, os valores inválidos aparentam ter sido informados sem casas decimais. Vamos verificar quantos valores estão acima de 10 reais:

In [0]:
preços.query("`VALOR VENDA` > 10")

,CNPJ,NOME,ENDEREÇO,COMPLEMENTO,BAIRRO,MUNICÍPIO,UF,PRODUTO,VALOR VENDA,DATA CADASTRO
200,4528732000371,POSTO NOVA PIRAI,"RUA XV DE NOVEMBRO,36",GALPAO,CENTRO,PIRAI,RJ,Gasolina C Comum,499.0,2019-08-16 11:39:00
201,4528732000371,POSTO NOVA PIRAI,"RUA XV DE NOVEMBRO,36",GALPAO,CENTRO,PIRAI,RJ,Etanol,389.0,2019-08-16 11:39:00
202,4528732000371,POSTO NOVA PIRAI,"RUA XV DE NOVEMBRO,36",GALPAO,CENTRO,PIRAI,RJ,GNV,349.0,2019-08-16 11:39:00
203,4528732000371,POSTO NOVA PIRAI,"RUA XV DE NOVEMBRO,36",GALPAO,CENTRO,PIRAI,RJ,Diesel S500,349.0,2019-08-16 11:39:00
204,4528732000371,POSTO NOVA PIRAI,"RUA XV DE NOVEMBRO,36",GALPAO,CENTRO,PIRAI,RJ,Diesel S10,359.0,2019-08-16 11:39:00
208,4624593000118,C JUSTINIANO ROCHA & CIA LTDA.,"AVENIDA LUCIO PEREIRA LUZ,790",NaN,CENTRO,LUCIARA,MT,Gasolina C Comum,550.0,2019-06-14 10:28:00
209,4624593000118,C JUSTINIANO ROCHA & CIA LTDA.,"AVENIDA LUCIO PEREIRA LUZ,790",NaN,CENTRO,LUCIARA,MT,Etanol,360.0,2019-06-14 10:28:00
210,4624593000118,C JUSTINIANO ROCHA & CIA LTDA.,"AVENIDA LUCIO PEREIRA LUZ,790",NaN,CENTRO,LUCIARA,MT,Diesel S500,445.0,2019-06-14 10:28:00
211,4624593000118,C JUSTINIANO ROCHA & CIA LTDA.,"AVENIDA LUCIO PEREIRA LUZ,790",NaN,CENTRO,LUCIARA,MT,Diesel S10,455.0,2019-06-14 10:28:00
442,10750422000138,POSTO PONTO CERTO,"AVENIDA GOV. FERNANDO GUILHON,1300",NaN,SAO LUIZ II,CONCEICAO DO ARAGUAIA,PA,Gasolina C Comum,505.0,2019-05-13 16:43:00


Por serem poucas observações, podemos removê-las sem prejuízo à base:

In [0]:
preços = preços.query("`VALOR VENDA` <= 10")

In [0]:
px.box(preços, x="PRODUTO", y="VALOR VENDA")

Curiosamente, agora vemos que também há outliers abaixo dos boxplots. Vamos investigar esses casos:

In [0]:
preços.query("`VALOR VENDA` <= 1")

,CNPJ,NOME,ENDEREÇO,COMPLEMENTO,BAIRRO,MUNICÍPIO,UF,PRODUTO,VALOR VENDA,DATA CADASTRO
249,5037623000152,AUTO POSTO TIO SAM LTDA,"RODOVIA BR 163,S/N","KM 20,5",ZONA RURAL,MUNDO NOVO,MS,GNV,0.001,2019-09-20 13:24:00
360,8355825000130,POSTO DE COMBUSTÍVEIS 214 SUL,SETOR SHC/SUL SQ 214 BLOCO A PAG - LOJA DE CON...,NaN,ASA SUL,BRASILIA,DF,GNV,1.000,2018-06-26 10:59:00
361,8355825000130,POSTO DE COMBUSTÍVEIS 214 SUL,SETOR SHC/SUL SQ 214 BLOCO A PAG - LOJA DE CON...,NaN,ASA SUL,BRASILIA,DF,Diesel S500,1.000,2018-06-26 10:59:00
491,11664743000182,POSTO UNIVERSITÁRIO,"RUA FREI GABRIEL,897",TERREO,UNIVERSITARIO,LAGES,SC,GNV,0.001,2019-03-29 17:00:00
659,19292157000166,VALTER GAVASSA COMBUSTIVEIS LTDA,"AVENIDA AFIF JOSE ABDO,37",NaN,RESIDENCIAL PORTAL DA PEROLA II,BIRIGUI,SP,GNV,1.000,2018-11-10 17:15:00
719,22794128000298,POSTO MIMIM II,"AVENIDA SETE DE SETEMBRO,623",LOJA 01,CENTRO,ITAJAI,SC,Gasolina C Comum,1.000,2019-10-16 17:50:00
720,22794128000298,POSTO MIMIM II,"AVENIDA SETE DE SETEMBRO,623",LOJA 01,CENTRO,ITAJAI,SC,Etanol,1.000,2019-10-16 17:50:00
721,22794128000298,POSTO MIMIM II,"AVENIDA SETE DE SETEMBRO,623",LOJA 01,CENTRO,ITAJAI,SC,GNV,1.000,2019-10-16 17:50:00
722,22794128000298,POSTO MIMIM II,"AVENIDA SETE DE SETEMBRO,623",LOJA 01,CENTRO,ITAJAI,SC,Diesel S10,1.000,2019-10-16 17:50:00


Novamente, parece seguro remover estes casos:

In [0]:
preços = preços.query("`VALOR VENDA` > 1")

In [0]:
px.box(preços, x="PRODUTO", y="VALOR VENDA")

## Preço médio do combustível

Agora que limpamos os dados, vamos visualizar o preço médio por tipo de combustível:

In [0]:
preço_produto = preços.pivot_table(index="PRODUTO", values="VALOR VENDA")
preço_produto.head()

,VALOR VENDA
PRODUTO,
Diesel S10,3.717703
Diesel S500,3.657632
Etanol,3.417888
GNV,3.248063
Gasolina C Comum,4.570035


In [0]:
px.bar(preço_produto, x=preço_produto.index, y="VALOR VENDA", title='Preço médio por tipo de combustível')

Botemos refinar nossa análise analisando o preço da gasolina comum por estado:

In [0]:
preço_gasolina = preços.query("PRODUTO == 'Gasolina C Comum'")
gasolina_por_estado = preço_gasolina.pivot_table(index="UF", values='VALOR VENDA')
gasolina_por_estado.head()

,VALOR VENDA
UF,
AL,3.959000
AM,3.980000
BA,4.638111
CE,4.685222
DF,4.455333


Para gerar este gráfico a partir da tabela dinâmica acima, vamos informar que os índices das observações devem ser usados como valores para o eixo x:

In [0]:
px.bar(gasolina_por_estado, x=gasolina_por_estado.index, y="VALOR VENDA", title = 'Preço médio da Gasolina comum por UF')

Podemos expandir essa análise para incluir todos os produtos considerados. Para isto, vamos usar um histograma, informando com o parâmetro `histfunc="avg"` que estamos interessados no valor médio. Note que, apesar de ser um gráfico com muitas informações, é possível selecionar quais produtos observar clicando na legenda mostrada no lado direito.

In [0]:
px.histogram(preços, x="UF", y="VALOR VENDA", color="PRODUTO", histfunc="avg",
             barmode="group", title='Distribuição de preços por combustível')

## Analisando a evolução do preço dos combustíveis

Os dados disponíveis na base que baixamos são referentes a vários meses distintos. Chamamos este tipo de dado de uma **série temporal**, ou série histórica. Podemos visualizar a evolução destas séries usando gráficos de linhas. Para isso, precisamos inicialmente gerar a série para o valor médio de cada produto por mês.

O primeiro passo é produzir uma característica contendo apenas os dados de ano e mês, para que tenhamos dados suficientes para uma agregação. Fazemos isso usando o método `.dt.to_period("M")` que características reconhecidas como data apresentam:

In [0]:
preços['MES'] = preços['DATA CADASTRO'].dt.to_period('M').astype(str)
preços.head()

,CNPJ,NOME,ENDEREÇO,COMPLEMENTO,BAIRRO,MUNICÍPIO,UF,PRODUTO,VALOR VENDA,DATA CADASTRO,MES
0,62780000102,AUTO POSTO PARATI,"AVENIDA FILINTO MULLER,645",NaN,CENTRO,TRES LAGOAS,MS,Gasolina C Comum,4.436,2018-06-28 17:49:00,2018-06
1,62780000102,AUTO POSTO PARATI,"AVENIDA FILINTO MULLER,645",NaN,CENTRO,TRES LAGOAS,MS,Etanol,3.482,2018-06-28 17:49:00,2018-06
2,62780000102,AUTO POSTO PARATI,"AVENIDA FILINTO MULLER,645",NaN,CENTRO,TRES LAGOAS,MS,Diesel S500,3.644,2018-06-28 17:49:00,2018-06
3,62780000102,AUTO POSTO PARATI,"AVENIDA FILINTO MULLER,645",NaN,CENTRO,TRES LAGOAS,MS,Diesel S10,3.734,2018-06-28 17:49:00,2018-06
4,300357000195,POSTO E TRANSPORTADORA PEGORARO,"RODOVIA BR 163,S/N",KM 786,ZONA RURAL,COXIM,MS,Gasolina C Comum,4.349,2018-09-12 19:06:00,2018-09


Um detalhe técnico do código acima é que o método `.dt.to_period("M")` gera um objeto do tipo `Period` atualmente incompatível com o gráfico de linhas da biblioteca Plotly. Por isso, pedimos para que o Pandas trate esta coluna como um texto usando o método `astype(str)`.

Agora que temos informação para cada mês, podemos gerar uma tabela dinâmica para visualizar a média por mês e produto:

In [0]:
preços_mês = preços.pivot_table(index="MES", columns="PRODUTO", values="VALOR VENDA")
preços_mês

PRODUTO,Diesel S10,Diesel S500,Etanol,GNV,Gasolina C Comum
MES,,,,,
2018-01,3.970000,3.890000,3.108000,NaN,5.122000
2018-03,3.619000,3.671500,3.599000,5.099000,4.911750
2018-04,3.694333,3.635333,2.973000,NaN,4.514333
2018-05,3.737000,3.647000,3.399000,2.899000,4.667400
2018-06,3.699333,3.573529,3.506625,2.197000,4.522105
2018-07,3.487478,3.410250,3.181100,2.743500,4.465407
2018-08,3.642308,3.575250,3.040000,NaN,4.547200
2018-09,3.591118,3.583000,3.445143,NaN,4.592706
2018-10,3.776667,3.876667,3.295800,NaN,5.067500


Note que o `DataFrame`a acima contém muitos valores faltantes para o GNV. Assim, vamos descartar dados referentes a este produto:

In [0]:
preços_mês = preços_mês.drop("GNV", axis=1)
preços_mês.head()

PRODUTO,Diesel S10,Diesel S500,Etanol,Gasolina C Comum
MES,,,,
2018-01,3.970000,3.890000,3.108000,5.122000
2018-03,3.619000,3.671500,3.599000,4.911750
2018-04,3.694333,3.635333,2.973000,4.514333
2018-05,3.737000,3.647000,3.399000,4.667400
2018-06,3.699333,3.573529,3.506625,4.522105


Apesar de útil para exploração de dados, o formato da tabela dinâmica acima não é adequado para a produção de um gráfico de linhas com a biblioteca Plotly. O código abaixo converte o formato *wide* acima em um formato *longo*, usando os métodos `stack` e `reset_index`:

In [0]:
preços_mês = preços_mês.stack().reset_index(name="VALOR VENDA")
preços_mês.head()

,MES,PRODUTO,VALOR VENDA
0,2018-01,Diesel S10,3.970
1,2018-01,Diesel S500,3.890
2,2018-01,Etanol,3.108
3,2018-01,Gasolina C Comum,5.122
4,2018-03,Diesel S10,3.619


Agora podemos investigar a evolução dos preços médios mensais de cada produto ao longo do período abrangido pela base de dados. Assim como no caso do histograma, podemos selecionar as séries que desejamos analisar interagindo com a legenda do gráfico:

In [0]:
px.line(preços_mês, x="MES", y="VALOR VENDA", color="PRODUTO",
        title='Evolução de preços por mês de observação (média do mês)')